<div style="text-align: center"><img width=300px src="https://www.uni-koblenz-landau.de/de/koblenz/logo.png"></div>


# VTNA



In [17]:

import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import seaborn as sns
import matplotlib.animation as animation
from pylab import *

from ipywidgets import Button, HBox, VBox
from IPython.display import display, HTML





In [18]:
dpi = 100

def ani_frame():
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    im = ax.imshow(rand(300,300),cmap='gray',interpolation='nearest')
    im.set_clim([0,1])
    fig.set_size_inches([5,5])


    tight_layout()


    def update_img(n):
        tmp = rand(300,300)
        im.set_data(tmp)
        return im

    #legend(loc=0)
    ani = animation.FuncAnimation(fig,update_img,300,interval=30)
    writer = animation.writers['ffmpeg'](fps=30)

    ani.save('demo.mp4',writer=writer,dpi=dpi)
    return ani

In [19]:
ht9_df = pd.read_csv('data/ht09_contact_list.dat', sep='\t', header=None, names=['timestamp', 'node1', 'node2'])
time_sec = ht9_df['timestamp'].max() - ht9_df['timestamp'].min()
def safe_div(x,y):
    if y == 0:
        return 0
    return x / y

update_delta_sec = 20



In [20]:
ht9_aggr_df = ht9_df.groupby(['node1', 'node2'], as_index=False).count().rename(columns={'timestamp': 'observations'})
ht9_aggr_net = nx.Graph()
ht9_aggr_net.add_edges_from((t[1], t[2], {'observations': t[3]}) for t in ht9_aggr_df.itertuples())
ht9_aggr_deg_cent = list(nx.degree_centrality(ht9_aggr_net).values())
ht9_aggr_clo_cent = list(nx.closeness_centrality(ht9_aggr_net).values())
ht9_aggr_layout = nx.spring_layout(ht9_aggr_net, k=2.0/np.sqrt(len(ht9_aggr_net.nodes)))

In [21]:
# Compute layout of nodes for aggregated network.
ht9_aggr_layout = nx.spring_layout(ht9_aggr_net, k=safe_div(2.0,np.sqrt(len(ht9_aggr_net.nodes))))
initial_time = ht9_df['timestamp'].min()

plt.figure(figsize=(8, 8))
fig = plt.gcf()
axes = plt.gca()
fig_num = fig.number


def draw_current_temp_aggr(current_time):
    ht9_tem_agg_net = nx.Graph()
    ht9_tem_agg_net.add_edges_from([(t[2], t[3]) for t in ht9_df[ht9_df['timestamp'] == current_time].itertuples()])
    
    plt.figure(fig_num, figsize=(8, 8))
    fig = plt.gcf()
    axes = plt.gca()
    axes.set_xlim((-1.2, 1.2))
    axes.set_ylim((-1.2, 1.2))
    nx.draw_networkx(ht9_tem_agg_net, ht9_aggr_layout, ax=axes, with_labels=False, node_size=75, node_color='green')
    axes.set_title('time = {}'.format(current_time))
    plt.show()
    
time_slider = widgets.IntSlider(
    value=initial_time,
    min=initial_time,
    description='Timeline: ',
    max=ht9_df['timestamp'].max(),
    step=update_delta_sec,
    continuous_update=False
)


#widgets.interact(draw_current_temp_aggr, current_time=time_slider)
plot = widgets.interactive(draw_current_temp_aggr, current_time=time_slider)

In [22]:


play = widgets.Play(
#     interval=10,
    value=50,
    min=0,
    max=ht9_df['timestamp'].max(),
    step=20,
    description="Press play",
    disabled=False
)

widgets.jslink((play, 'value'), (time_slider, 'value'))
widgets.VBox([plot, play])

VBox(children=(interactive(children=(IntSlider(value=20, continuous_update=False, description='Timeline: ', max=212360, min=20, step=20), Output()), _dom_classes=('widget-interact',)), Play(value=50, description='Press play', max=212360, step=20)))

In [23]:

exportbutton = widgets.Button(
    value=False,
    description='Convert',
    disabled=False,
    button_style='', 
    tooltip='Download animation',
    icon='download'
)

In [24]:
filetyperadios = widgets.RadioButtons(
    options=['Mp4', 'Gif'],
    description='Filetype:',
    disabled=False
)

In [25]:
box = HBox([filetyperadios, exportbutton])
settings = HBox([filetyperadios, exportbutton])

In [26]:
tab_nest = widgets.Tab()
tab_nest.children = [box, settings]
tab_nest.set_title(0, 'Export')
tab_nest.set_title(1, 'Settings')
tab_nest

Tab(children=(HBox(children=(RadioButtons(description='Filetype:', options=('Mp4', 'Gif'), value='Mp4'), Button(description='Convert', icon='download', style=ButtonStyle(), tooltip='Download animation'))), HBox(children=(RadioButtons(description='Filetype:', options=('Mp4', 'Gif'), value='Mp4'), Button(description='Convert', icon='download', style=ButtonStyle(), tooltip='Download animation')))), _titles={'0': 'Export', '1': 'Settings'})

In [27]:
# To Install ffmpeg: conda install -c conda-forge ffmpeg
'''
ani_frame()
'''


plt.figure(fig_num, figsize=(8, 8))
fig = plt.gcf()
axes = plt.gca()
axes.set_xlim((-1.2, 1.2))
axes.set_ylim((-1.2, 1.2))


def draw_current_temp_aggr(current_time):
    current_time = current_time*20
    fig = plt.gcf()
    ht9_tem_agg_net = nx.Graph()
    ht9_tem_agg_net.add_edges_from([(t[2], t[3]) for t in ht9_df[ht9_df['timestamp'] == current_time].itertuples()])
    axes.set_title('time = {}'.format(current_time))
    nx.draw_networkx(ht9_tem_agg_net, ht9_aggr_layout, ax=axes, with_labels=False, node_size=75, node_color='green')



#ani = animation.FuncAnimation(fig,draw_current_temp_aggr,5000/20,interval=30)
#writer = animation.writers['ffmpeg'](fps=30)

#ani.save('demo.mp4',writer=writer,dpi=dpi)


In [6]:
from IPython.display import FileLink, FileLinks
FileLinks('.')

./
  demo1.mp4
  notebook.tex
  vtna.ipynb
.\.ipynb_checkpoints/
  vtna-checkpoint.ipynb
.\data/
  ht09_contact_list.dat

In [29]:
html = '<a download="filename" href="/files/demo1.mp4" target="_blank">Download</a>'
HTML(html)



In [13]:
#https://github.com/peteut/ipython-file-upload
#https://github.com/ipython-contrib/jupyter_contrib_nbextensions

from IPython.display import display
import fileupload

uploader = fileupload.FileUploadWidget()

def _handle_upload(change):
    w = change['owner']
    with open(w.filename, 'wb') as f:
        f.write(w.data)
    print('Uploaded `{}` ({:.2f} kB)'.format(
        w.filename, len(w.data) / 2**10))

uploader.observe(_handle_upload, names='data')

display(uploader)

FileUploadWidget(label='Browse', _dom_classes=('widget_item', 'btn-group'))

Uploaded `Liederliste.txt` (3.22 kB)
Uploaded `ht09_contact_list.dat.gz` (65.88 kB)


FileUploadWidget(label='Browse', _dom_classes=('widget_item', 'btn-group'))

In [31]:
# http://chris-said.io/2016/02/13/how-to-make-polished-jupyter-presentations-with-optional-code-visibility/
HTML('''
<script>
    code_show=true; 
    function code_toggle() {
         if (code_show){
             $('div.input').hide();
         } else {
             $('div.input').show();
         }
         code_show = !code_show
    } 
    $( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.
''')